In [14]:
import os

new_directory = '/home/franciscoperez/Documents/GitHub/CNN-PELSVAE2/cnn-pels-vae/'
os.chdir(new_directory)

import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F


from src.focalloss import  FocalLossMultiClass as focal_loss
import src.utils as utils 
from src.cnn import initialize_masks, train_one_epoch_alternative, create_dataloader, setup_environment, initialize_optimizers
from src.sampler.getbatch import SyntheticDataBatcher

from src.utils import get_data
import yaml 
import numpy as np
from typing import Union, Tuple, Optional, Any, Dict, List

from sklearn.utils.class_weight import compute_class_weight
import torch.optim as optim

with open('src/regressor.yaml', 'r') as file:
    config_file: Dict[str, Any] = yaml.safe_load(file)
vae_model: str =   config_file['model_parameters']['ID']  
data_sufix: str =   config_file['model_parameters']['sufix_path']  

with open('src/nn_config.yaml', 'r') as file:
    nn_config = yaml.safe_load(file)
    
PP = utils.load_pp_list(vae_model)
prior = False
create_samples = False
wandb_active = False
N_LAYERS = 4
opt_method= "oneloss"

In [19]:
class CNN(nn.Module):
    def __init__(self, num_classes: int = 2, layers = 2) -> None:
        super(CNN, self).__init__()

        self.layers = layers
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=8, kernel_size=6, stride=1)
        init.xavier_uniform_(self.conv1.weight)  

        self.bn1 = nn.BatchNorm1d(8)
        self.pool1 = nn.MaxPool1d(2)
        
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=6, stride=1)
        init.xavier_uniform_(self.conv2.weight)  

        self.bn2 = nn.BatchNorm1d(16)
        self.pool2 = nn.MaxPool1d(2)

        if self.layers > 2: 
            self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=6, stride=1)
            init.xavier_uniform_(self.conv3.weight)  
            self.bn3 = nn.BatchNorm1d(32)
            self.pool3 = nn.MaxPool1d(2)  
        
        if self.layers > 3: 
            self.conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=6, stride=1)
            init.xavier_uniform_(self.conv4.weight)  
            self.bn4 = nn.BatchNorm1d(64)
            self.pool4 = nn.MaxPool1d(2)  

        if self.layers == 2:
            self.fc1 = nn.Linear(1136, 200)
            init.xavier_uniform_(self.fc1.weight)  
        elif self.layers == 3:
            self.fc1 = nn.Linear(1056, 200)
            init.xavier_uniform_(self.fc1.weight) 
        elif self.layers == 4:
            self.fc1 = nn.Linear(896, 200)
            init.xavier_uniform_(self.fc1.weight) 

        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(200, num_classes)
        init.xavier_uniform_(self.fc2.weight)  

        self.dropout2 = nn.Dropout(0.2)



    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.tanh(x)
        x = self.pool1(x)        

        
        x = self.conv2(x)
        x = self.bn2(x)
        
        x = F.tanh(x)
        
        x = self.pool2(x)

        if self.layers == 3: 
            x = self.conv3(x)
            x = self.bn3(x)
            x = F.tanh(x)
            x = self.pool3(x)
            
        if self.layers == 4: 
            x = self.conv3(x)
            x = self.bn3(x)
            x = F.tanh(x)
            x = self.pool3(x)
            
            x = self.conv4(x)
            x = self.bn4(x)
            x = F.tanh(x)
            x = self.pool4(x)
        
        x = x.view(x.size(0), -1)  
        x = self.fc1(x)
        x = F.tanh(x)
        #x = self.dropout1(x)
        
        x = self.fc2(x)
        #x = self.dropout2(x)
        
        if (nn_config['training']['loss']=='NLLLoss') or (nn_config['training']['loss']=='focalLoss'):
            return F.log_softmax(x, dim=1)  
        else: 
            return x
        
def setup_model(num_classes: int, device: torch.device, show_architecture: bool = True) -> nn.Module:

    print('----- model setup --------')
    model = CNN(num_classes=num_classes, layers = N_LAYERS)
    if torch.cuda.is_available():
        model = model.to(device)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    if show_architecture:
        print("Model Architecture:")
        print(model)

    return model

In [20]:
device = setup_environment()

print('------ Data loading -------------------')
print('mode: ', nn_config['data']['mode_running'], nn_config['data']['sample_size'])
x_train, x_test, y_train, y_test, x_val, y_val, \
label_encoder, y_train_labeled, y_test_labeled = get_data(nn_config['data']['sample_size'], 
                                                          nn_config['data']['mode_running'])

print(label_encoder)
print('Training set')
classes = np.unique(y_train_labeled.numpy())
num_classes = len(classes)
print('num_classes: ', num_classes)
    
model = setup_model(num_classes, device)
class_weights = compute_class_weight('balanced', np.unique(y_train_labeled.numpy()), y_train_labeled.numpy())
class_weights = np.sqrt(class_weights)
class_weights = torch.tensor(class_weights).to(device, dtype=x_train.dtype)

print(class_weights)
if nn_config['training']['loss']=='CrossEntropyLoss':
    criterion = nn.CrossEntropyLoss(weight=class_weights) 
    criterion_synthetic_samples = nn.CrossEntropyLoss(weight=class_weights) 
elif nn_config['training']['loss']=='NLLLoss': 
    criterion = nn.NLLLoss(weight=class_weights) 
    criterion_synthetic_samples = nn.NLLLoss(weight=class_weights) 
elif  nn_config['training']['loss']=='focalLoss':
    criterion = focal_loss(alpha=class_weights, gamma=2)
    criterion_synthetic_samples = focal_loss(alpha=class_weights, gamma=2)      
else: 
    raise('The required loss is not supported, '+ nn_config['training']['loss'])

training_data = utils.move_data_to_device((x_train, y_train), device)
val_data = utils.move_data_to_device((x_val, y_val), device)
testing_data = utils.move_data_to_device((x_test, y_test), device)

best_val_loss = float('inf')
best_f1_val = 0
harder_samples = True
no_improvement_count = 0
train_loss_values = []
val_loss_values = []
train_accuracy_values = []
val_accuracy_values = []
counter = 0

epochs = nn_config['training']['epochs']
patience =  nn_config['training']['patience']
batch_size = nn_config['training']['batch_size']
repetitions = nn_config['training']['repetitions']
sinthetic_samples_by_class = nn_config['training']['sinthetic_samples_by_class']
threshold_acc_synthetic = nn_config['training']['threshold_acc_synthetic'] 
beta_decay_factor= nn_config['training']['beta_decay_factor'] 
beta_initial= nn_config['training']['beta_initial'] 
EPS= nn_config['training']['EPS'] 
base_learning_rate= nn_config['training']['base_learning_rate'] 
scaling_factor= nn_config['training']['scaling_factor'] 

alpha = nn_config['training']['alpha']

train_dataloader = create_dataloader(training_data, batch_size)
val_dataloader = create_dataloader(val_data, batch_size)
test_dataloader = create_dataloader(testing_data, batch_size)

beta_actual = beta_initial

optimizer1, optimizer2, locked_masks, locked_masks2 = initialize_optimizers(model, opt_method= opt_method,
                                                                            EPS=EPS, base_learning_rate=base_learning_rate, 
                                                                            scaling_factor=scaling_factor)

batcher = SyntheticDataBatcher(PP = PP, vae_model=vae_model, n_samples=sinthetic_samples_by_class, 
                                seq_length = x_train.size(-1), prior=prior)

priorization = True
for epoch in range(2):
    print(opt_method, create_samples, harder_samples)
    if opt_method=='twolosses' and create_samples and harder_samples: 
        if epoch>10 and priorization:
            ranking, _ = get_dict_class_priorization(model, train_dataloader)
            dict_priorization = {}
            ranking_penalization = 2

            for o in ranking:
                dict_priorization[label_encoder[o]] =  int(sinthetic_samples_by_class*ranking_penalization)
                if ranking_penalization>0.5:
                    ranking_penalization = ranking_penalization/2

            synthetic_data_loader = batcher.create_synthetic_batch(b=beta_actual, 
                                                                wandb_active=wandb_active, 
                                                                samples_dict = dict_priorization)
        else: 
            synthetic_data_loader = batcher.create_synthetic_batch(b=beta_actual, 
                                                wandb_active=wandb_active, 
                                                samples_dict = None)

        beta_actual = beta_actual*beta_decay_factor
        harder_samples = False
    elif  opt_method=='twolosses' and create_samples: 
        print("Using available synthetic data")
    else:
        print("Skipping synthetic sample creation")
        synthetic_data_loader = None

    running_loss, model, val_loss = train_one_epoch_alternative(model, criterion, optimizer1, train_dataloader, val_dataloader, device, 
                                    mode = opt_method, 
                                    criterion_2= criterion_synthetic_samples, 
                                    dataloader_2 = synthetic_data_loader,
                                    optimizer_2 = optimizer2, locked_masks2 = locked_masks2, 
                                    locked_masks = locked_masks, repetitions = repetitions)

CUDA active: True
------ Data loading -------------------
mode:  load 400000
--------------------------------------------------
MODE DATA:  load
Loading light curves
loaded files
training data shape:  (205659, 300, 2)
testing data shape:  (19484, 300, 2)
Loaded shape training set:  (205659, 300, 2)
Loaded shape testing set:  (19484, 300, 2)
Modified shape training set:  (205659, 2, 300)
Modified shape testing set:  (19484, 2, 300)
Label to Number Mapping:
CEP: 0
DSCT: 1
ECL: 2
LPV: 3
RRLYR: 4
T2CEP: 5
['CEP', 'DSCT', 'ECL', 'LPV', 'RRLYR', 'T2CEP']
Training set
num_classes:  6
----- model setup --------
Using 2 GPUs!
Model Architecture:
DataParallel(
  (module): CNN(
    (conv1): Conv1d(2, 8, kernel_size=(6,), stride=(1,))
    (bn1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv1d(8, 16, kernel_size=(6,), stride=(1,))
    (bn2): BatchNorm1d(16, eps=